#Public Highlights
##Python Code
* **About the Data**
    * Got data from userhighlights not teamhighlights
    * All userhighlights = 7286510
    * Userhighlights with views = 6927719
    * *5% of deleted because data no v for view*
* **More information**
    * mean: 24.9429066046
    * stdev: 505.207633089
    * max: 1056352.0
    * min: 0.0
* **Zero Views**
    * 1134396 highlights
        * *16% of all userhighlights*
    * 99.8% of the 0 views are non premium highlights
        * 18% of all non premium highlights are 0 views
    * .3% of all premium highlights are 0 views
        * .2 % of the 0 views are non premium highlights
    

In [11]:
import json
import numpy as np

In [ ]:
sqlc = HiveContext(sc)
user_highlights = sc.textFile("s3n://ds-fulla/mongo/20150507/monolith/highlights/userhighlights/").map(json.loads)
highlight_views = user_highlights.filter(lambda x: 'v' in x.keys())
all_views = highlight_views.map(lambda x: x['v']).reduce(lambda a,b: a+b)
zero_views = highlight_views.filter(lambda x: x['v']==0)
highlight_views.map(lambda x: x['v']).stats()

##Premium
* Premium highlights = 636552 
* Premium highlight views = 114767406  
* Premium view information
    * mean: 180.295413415
    * median: 73.0
        * from a fulla sample
    * stdev: 1652.72230355
    * max: 1056352.0
    * min: 0.0

In [ ]:
yp = highlight_views.filter(lambda x: x['p'])
zero_yp = zero_views.filter(lambda x: x['p']).count()
yp_views = yp.map(lambda x: x['v']).reduce(lambda a,b: a+b)
yp.map(lambda x: x['v']).stats()

In [ ]:
#Find yp median
sqlc.sql("ADD JAR s3://ds-fulla/scripts/bin/jsonserde.jar")
query1 = """SELECT views FROM mono_highlights.userhighlights WHERE ispremium = true"""
query1_count = sqlc.sql(query1).map(lambda x: x[0]).filter(lambda x: x != None).count()
query1_views = sqlc.sql(query1).map(lambda x: x[0]).filter(lambda x: x != None).collect()
query1_median = np.median(query1_views)